In [65]:
import os, sys, csv, time
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import numpy as np
import networkx as nx
from itertools import izip
from Bio.Alphabet import IUPAC

In [2]:
os.listdir('.')

['genbank',
 'fastas',
 'backup',
 '.ipynb_checkpoints',
 'concat',
 'Header-Reprocessing.ipynb',
 'Graph_Construction.ipynb',
 'alignments_test']

In [3]:
r6_modified = list(SeqIO.parse('fastas/R6_new.mfa', 'fasta'))
r6_original = list(SeqIO.parse('backup/R6.mfa', 'fasta'))

In [4]:
print r6_modified[0].id
print r6_original[0].id

R6.1_1
lcl|AE007317.1_prot_AAK98805.1_1


In [5]:
def dict_maker(modified_files, original_files):
    mod_list = []
    orig_list = []
    for index in xrange(len(modified_files)):
        mod_rec = list(SeqIO.parse(modified_files[index], 'fasta'))
        orig_rec = list(SeqIO.parse(original_files[index], 'fasta'))
        for index2 in xrange(len(mod_rec)):
                mod_list.append(mod_rec[index2].id)
                orig_list.append(orig_rec[index2].id)
    return dict(izip(mod_list, orig_list)), mod_list

In [6]:
#Paths to fasta and backup files.
fastapath = '/home/jacob/Documents/CMU/Research/Genomes/fastas/'
backup = '/home/jacob/Documents/CMU/Research/Genomes/backup/'


modified_files = [fastapath + 'ATCC700669_new.mfa', fastapath + 'D39_new.mfa', fastapath + 'G54_new.mfa', \
                  fastapath + 'TIGR4_new.mfa', fastapath + 'R6_new.mfa']

original_files = [backup + 'ATCC700669.mfa', backup + 'D39.mfa', backup + 'G54.mfa', \
                 backup + 'TIGR4.mfa', backup + 'R6.mfa']

nucl_fastas = os.listdir(fastapath + 'nucl/')

headers_dict, mod_list = dict_maker(modified_files, original_files) 

def reverse(dict):
    return {v: k for k, v in dict.iteritems()}

inv_head_dict = reverse(headers_dict)

In [7]:
with open('concat/mcl_clusters', 'r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    clusters = list(reader)

def cluster_dict_maker(cluster_file, mod_list):
    mod_comp = [[i] for i in mod_list]
    for index, cluster in enumerate(cluster_file):
        for element in cluster:
            mod_comp[mod_list.index(element)].append(index)
    return dict(mod_comp)

In [8]:
cluster_dict = cluster_dict_maker(clusters, mod_list)
gb_dict = {"AP200":"CP002121",\
                "670-6B":"CP002176",\
                "70585":"CP000918",\
                "ATCC700669":"FM211187.1",\
                "CGSP14":"CP001033",\
                "D39":"CP000410.1",\
                "G54":"CP001015.1",\
                "Hu19A":"CP000936",\
                "INV104":"FQ312030",\
                "INV200":"FQ312029",\
                "JJA":"CP000919",\
                "OXC141":"FQ312027",\
                "P1031":"CP000920",\
                    "SPN032672":"NC_021003",\
                "SPN033038":"NC_021004",\
                "SPN034183":"FQ312043",\
                "SPN994038":"FQ312041",\
                "SPN994039":"FQ312044",\
                "SPN034156":"FQ312045",\
                "T19F":"CP000921.1",\
                "TIGR4":"AE005672.3",\
                "R6":"NC_003098.1"}


In [9]:
gbks = '/home/jacob/Documents/CMU/Research/Genomes/genbank/'
ATCC = list(SeqIO.parse(gbks+'ATCC700669.gb', 'genbank'))

for filename in os.listdir(gbks):
    if filename.split('.')[-1] == 'gb':
        blahblah = list(SeqIO.parse(gbks+filename, 'genbank'))
        print filename
        print(blahblah[0].id)

TIGR4.gb
AE005672.3
D39.gb
CP000410.1
G54.gb
CP001015.1
ATCC700669.gb
FM211187.1
R6.gb
NC_003098.1


## So shit with the headers is mighty confusing. Sometimes there's a '.1' or '.3' at the end, sometimes it's a bunch of irrelevant alphanumeric logorrhea, sometimes it's my simplified headers. So now I have to make a way to standardize the protein IDs.

## Retroactive note: I just used the dictionary I made for another script. Someday I will have my revenge, NCBI.

In [10]:
feats = [feat for feat in ATCC[0].features if feat.type == "CDS"]
len(feats)

2135

In [11]:
def strand_switch(start, end):
    #Compensate for strand ridiculousness
    if end < start:
        return end, start
    else:
        return start, end


def graph_constructor(genbank, cluster_dict, gb_dict):
    genome_id = gb_dict[genbank[0].id] #Get the non-bullshit version of the ID, thanks genbank
    feats = [feat for feat in genbank[0].features if feat.type == "CDS"]
    cluster_assignments = []
    G = nx.Graph()
    edge_locations = []
    
    for index, feat in enumerate(feats): #Make list of cluster memberships, distances to proximal genes
        if index == len(feats)-1:
            break
        cluster_assignments.append([])
        
        if genome_id + '.1_' + str(index+1) in cluster_dict:
            cluster_id = cluster_dict[genome_id + '.1_' + str(index+1)]
        elif genome_id + '.1_prot_' + str(index+1) in cluster_dict:
            cluster_id = cluster_dict[genome_id + '.1_prot_' + str(index+1)]
        else:
            cluster_id = cluster_dict[genome_id + '.2_' + str(index+1)]
        cluster_assignments[-1].append(cluster_id) #Make sure this accesses right
        
        cur_start = feat.location.start
        cur_end = feat.location.end
        cur_strand = feat.location.strand
        cur_feat = feat
        start, end = strand_switch(cur_start, cur_end)
        
        next_start = feats[index+1].location.start
        next_end = feats[index+1].location.end
        next_strand = feats[index+1].location.strand
        next_feat = feats[index+1]
        next_start, next_end = strand_switch(next_start, next_end) 
        if genome_id + '.1_' + str(index+2) in cluster_dict:
            next_cluster_id = cluster_dict[genome_id + '.1_' + str(index+2)]
        elif genome_id + '.1_prot_' + str(index+2) in cluster_dict:
            next_cluster_id = cluster_dict[genome_id + '.1_prot_' + str(index+2)]
        else: 
            cluster_id = cluster_dict[genome_id + '.2_' + str(index+2)]
        if next_cluster_id not in G.nodes():
            #If the next gene belongs to an unseen cluster, add a node for it
            G.add_node(next_cluster_id)
        
        G.add_edge(cluster_id, next_cluster_id, weight=next_start-end)
        edge_locations.append([genome_id, cur_end, next_start, abs(cur_end - next_start), cur_feat, next_feat])
        
    return G, cluster_assignments, edge_locations

In [12]:
#Testing!
ATCC_Graph, ATCC_clusts, ATCC_edgelocations = graph_constructor(ATCC, cluster_dict, reverse(gb_dict))

## Awesome. So the graph generator works (whether or not it works infallibly is yet to be determined); now I have to generate all the graphs, then find a way to remove edges that don't exist in each graph.

In [13]:
ordered_file_list = []
graph_list = []
cluster_lists = []
bed_lists = [] #Will contain all genome ID/insert position info

def edge_subgraph(mg_edgelist, graph_list):
    for graph in graph_list:
        cur_edgelist = list(graph.edges_iter())
        for edge in mg_edgelist:
            if edge not in cur_edgelist:
                del mg_edgelist[mg_edgelist.index(edge)]
    return mg_edgelist

def induce_subgraph_from_edgelist(G, edge_list, ref_back=True):
    #Ripped from https://stackoverflow.com/questions/16150557/networkxcreating-a-subgraph-induced-from-edges
    sub_nodes = list({y for x in edge_list for y in x[0:2]})
    edge_list_no_data = [edge[0:2] for edge in edge_list]
    assert all([e in G.edges() for e in edge_list_no_data])

    if ref_back:
        G_sub = G.subgraph(sub_nodes)
        for edge in G_sub.edges():
            if edge not in edge_list_no_data:
                G_sub.remove_edge(*edge)
    else:
        G_sub = G.subgraph(sub_nodes).copy()
        for edge in G_sub.edges():
            if edge not in edge_list_no_data:
                G_sub.remove_edge(*edge)

    return G_sub

for filename in os.listdir(gbks):
    if filename.split('.')[-1] == 'gb':
        ordered_file_list.append(filename.split('.')[0])
        genbank = list(SeqIO.parse(gbks+filename, 'genbank'))
        cur_graph, cur_clusts, cur_edges = graph_constructor(genbank, cluster_dict, reverse(gb_dict))
        graph_list.append(cur_graph)
        cluster_lists.append(cur_clusts)
        bed_lists.append(cur_edges)

    
#Make a metagraph cotaining each edge from all graphs
metagraph = nx.Graph()
for graph in graph_list:
    metagraph.add_edges_from(graph.edges(data=False))

#Make a masked array to track which edges are preserved; DON'T USE GRAPH_LIST[0]
masked_arr = np.zeros(len(metagraph.edges()))

meta_edgelist = list(metagraph.edges_iter())

#Named sub_graph instead of subgraph because there's a networkx method called .subgraph()
sub_graph = metagraph

for genome in graph_list:
    subgraph_edgelist = edge_subgraph(meta_edgelist, graph_list)
    #Remove self-edges; almost entirely adjacent transposons, which we will look at later
    subgraph_edgelist = [edge for edge in subgraph_edgelist if edge[0] != edge[1]]
    sub_graph = induce_subgraph_from_edgelist(sub_graph, subgraph_edgelist)

"""
#Remove self-edges in sub_graph (Transposons messing up my life; look at them later)
for (u,v,d) in sub_graph.edges_iter(data='weight'):
    if u == v:
        sub_graph.remove_edge(u,v)
"""

#Create filtered information based on edges
sg_edgelist = list(sub_graph.edges_iter()) 
edgelist_list = []

#Make list of edge lists for each genome graph
for graph in graph_list:
    edgelist_list.append(list(graph.edges_iter()))
                         
                         
bed_list = []

for index, edge in enumerate(meta_edgelist):
    if edge in sg_edgelist:
        masked_arr[index] = 1 #Index in masked array
        bed_list.append([])
        for genome_num in xrange(len(graph_list)):
            genome_id = ordered_file_list[genome_num].split('.')[0]
            edgeno = edgelist_list[genome_num].index(edge)
            #Append in format [Genome ID, edge coordinate 1, edge coordinate 2]
            #to add: upstream/downstream features? maybe protein IDs? We're gonna make this into a dataframe anyway
            bed_list[-1].append(bed_lists[genome_num][edgeno])


In [14]:
print("Subgraph edges:")
print(len(sub_graph.edges()))
for idx in xrange(len(graph_list)):
    print("graph_list[" + str(idx) + "] edges:")
    print(len(graph_list[idx].edges()))

Subgraph edges:
871
graph_list[0] edges:
2111
graph_list[1] edges:
1909
graph_list[2] edges:
2089
graph_list[3] edges:
2127
graph_list[4] edges:
1784


## HELLA

In [15]:
print bed_list[0]

[['TIGR4', ExactPosition(2853), ExactPosition(2863), 10, SeqFeature(FeatureLocation(ExactPosition(1716), ExactPosition(2853), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(2863), ExactPosition(3112), strand=1), type='CDS')], ['D39', ExactPosition(2657), ExactPosition(2721), 64, SeqFeature(FeatureLocation(ExactPosition(1520), ExactPosition(2657), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(2721), ExactPosition(2916), strand=1), type='CDS')], ['G54', ExactPosition(3112), ExactPosition(3195), 83, SeqFeature(FeatureLocation(ExactPosition(2917), ExactPosition(3112), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(3195), ExactPosition(4311), strand=1), type='CDS')], ['ATCC700669', ExactPosition(2842), ExactPosition(2906), 64, SeqFeature(FeatureLocation(ExactPosition(1705), ExactPosition(2842), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(2906), ExactPosition(3101), strand=1), type='CDS')], ['R6', ExactPosition(2657)

In [16]:
def check_for_insert(sublist):
    insert_sizes = []
    for genome_fragment in sublist:
        insert_sizes.append(genome_fragment[2] - genome_fragment[1])
    for i in xrange(len(insert_sizes)):
        for j in xrange(len(insert_sizes) - i):
            if abs(insert_sizes[i] - insert_sizes[j]) >= 150:
                return True
    return False

filtered_list = []
for sublist in bed_list:
    if check_for_insert(sublist):
        filtered_list.append(sublist)

In [17]:
print filtered_list[205]

[['TIGR4', ExactPosition(678099), ExactPosition(678710), 611, SeqFeature(FeatureLocation(ExactPosition(676623), ExactPosition(678099), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(678710), ExactPosition(680057), strand=-1), type='CDS')], ['D39', ExactPosition(579349), ExactPosition(579653), 304, SeqFeature(FeatureLocation(ExactPosition(577873), ExactPosition(579349), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(579653), ExactPosition(586340), strand=1), type='CDS')], ['G54', ExactPosition(625830), ExactPosition(626062), 232, SeqFeature(FeatureLocation(ExactPosition(625197), ExactPosition(625830), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(626062), ExactPosition(626413), strand=1), type='CDS')], ['ATCC700669', ExactPosition(695540), ExactPosition(695523), 17, SeqFeature(FeatureLocation(ExactPosition(694340), ExactPosition(695540), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(695523), ExactPosition(696225),

In [62]:
ATCC = list(SeqIO.parse(fastapath + 'nucl/ATCC700669.fasta', 'fasta'))
G54 = list(SeqIO.parse(fastapath + 'nucl/G54.fasta', 'fasta'))
D39 = list(SeqIO.parse(fastapath + 'nucl/D39.fasta', 'fasta'))
TIGR4 = list(SeqIO.parse(fastapath + 'nucl/TIGR4.fasta', 'fasta'))
R6 = list(SeqIO.parse(fastapath + 'nucl/R6.fasta', 'fasta'))
fasta_dict = {'ATCC700669':ATCC, 'G54':G54, 'D39':D    sys.exit()
39, 'TIGR4':TIGR4, 'R6':R6}

SyntaxError: invalid syntax (<ipython-input-62-787749756b0a>, line 6)

In [72]:
def sense(start, end):
    if start > end:
        start, end = end, start
    return start, end

def pull_seqs(sublist, protein=False):
    if protein:
        upstream_list = []
        downstream_list = []
        for index, element in enumerate(sublist):
            try:
                upstream_list.append(SeqRecord(seq=element[-1].qualifiers[('translation')][0], \
                                               id=element[0] + '_' + str(index), description="", \
                                               alphabet=IUPAC.protein))
            except TypeError:
                print index, element
            except:
                pass
            try:
                downstream_list.append(SeqRecord(seq=element[-1].qualifiers[('translation')][0], \
                                               id=element[0] + '_' + str(index), description="", \
                                                 alphabet=IUPAC.protein))
            except:
                pass
        return downstream_list, upstream_list
    else:
        seq_list = []
        for index, element in enumerate(sublist):
            start, end = sense(element[1], element[2])
            seq_list.append(SeqRecord(fasta_dict[element[0]][0].seq[start:end], \
                                      id=element[0] + '_' + str(index), description=""))
        return seq_list

def fix_null(list):
    new_list = []
    for element in list:
        if element != []:
            new_list.append(element)
    return new_list

for i in xrange(len(filtered_list)):
    seq_list = pull_seqs(filtered_list[i])
    downstream_list, upstream_list = pull_seqs(filtered_list[i], protein=True)
    print downstream_list[0]
    sys.exit()
    with open("seqgroup_" + str(i) + ".fasta", 'w') as output_handle:
        SeqIO.write(seq_list, output_handle, "fasta")
    with open("seqgroup_" + str(i) + "_downstream.faa", 'w') as output_handle:
        SeqIO.write(downstream_list, output_handle, "fasta")
    with open("seqgroup_" + str(i) + "_upstream.faa", 'w') as output_handle:
        SeqIO.write(upstream_list, output_handle, "fasta")
    os.system('mkdir run' + str(i) + ' && mv seqgroup_' + str(i) + '* run' + str(i))


0 ['TIGR4', ExactPosition(25416), ExactPosition(25417), 1, SeqFeature(FeatureLocation(ExactPosition(24972), ExactPosition(25416), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(25417), ExactPosition(25933), strand=1), type='CDS')]
1 ['D39', ExactPosition(4115), ExactPosition(4185), 70, SeqFeature(FeatureLocation(ExactPosition(2999), ExactPosition(4115), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(4185), ExactPosition(4755), strand=1), type='CDS')]
2 ['G54', ExactPosition(23826), ExactPosition(24076), 250, SeqFeature(FeatureLocation(ExactPosition(23709), ExactPosition(23826), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(24076), ExactPosition(24520), strand=1), type='CDS')]
3 ['ATCC700669', ExactPosition(27549), ExactPosition(27603), 54, SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(27219), ExactPosition(27549), strand=-1), FeatureLocation(ExactPosition(26407), ExactPosition(27220), strand=-1)], 'join'), type='CDS', 

IndexError: list index out of range

In [49]:
#print filtered_list[0][0][-1].qualifiers[('translation')]
downstream_list, upstream_list = pull_seqs(filtered_list[0], protein=True)
print downstream_list

[['MKIRGFELVSSFTDENLLPKRETAHAAGYDLKVAVRTVVAPGEIVLVPTGVKAYMQPTEVLYLYDRSSNPRKKGLVLINSVGVIDGDYYGNPGNEGHIFAQMKNITDQEVVLEVGERIVQAVFATFLIADGDAADGVRTGGFGSTGH'], ['MALTAGIVGLPNVGKSTLFNAITKAGAEAANYPFATIDPNVGMVEVPDERLQKLTEMITPKKTVPTTFEFTDIAGIVKGASKGEGLGNKFLANIREVDAIVHVVRAFDDENVMREQGREDAFVDPLADIDTINLELILADLESVNKRYARVEKMARTQKDKESVAEFNVLQKIKPVLEDGKSARTIEFTDEEQKVVKGLFLLTTKPVLYVANVDEDVVSEPDSIDYVKQIREFAATENAEVVVISARAEEEISELDDEDKKEFLEAIGLTESGVDKLTRAAYHLLGLGTYFTAGEKEVRAWTFKRGMKAPQAAGIIHSDFEKGFIRAVTMSYEDLVKYGSEKAVKEAGRLREEGKEYIVQDGDIMEFRFNV'], ['MPAFPNVVYGAKNQKFGAAGSLYNILTDERLNHLWRLK'], ['MKSWKNWLIKINYCVYKKKRKGRMRKFLIILLLPSFLTISKVVSTEKEVVYTSKEIYYLSQSDFGIYFREKLSSPMVYGEVPVYANEDLVVESGKLTPKTSFQITEWRLNKQGIPVFKLSNHQFIAADKRFLYDQSEVTPTIKKVWLESDFKLYNSPYDLKEVKSSLSAYSQVSIDKTMFVEGREFLHIDQAGWVAKESTSEEDNRMSKVQEMLSEKYQKDSFSIYVKQLTTGKEAGINQDEKMYAASVLKLSYLYYTQEKINEGLYQLDTTVKYVSAVNDFPGSYKPEGSGSLPKKEDNKEYSLKDLITKVSKESDNVAHNLLGYYISNQSDATFKSKMSAIMGDDWDPKEKLISSKMAGKFMEAIYNQNGFVLESLTKTDFDSQRIAKGVSVKVAHKIGDADEFKHDTGVVYADSPFILSI